<a href="https://colab.research.google.com/github/KunavazhagannSridar/Telium/blob/main/Telium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# @title
# Telium

import sys
import time
import random
from IPython.display import clear_output

# Global variables

num_modules = 21
module = 1
last_module = 0
possible_moves = []
alive = True
won = False
killed = False
power = 1000
fuel = 500
locked = 0
queen = 0
vent_shafts = []
info_panels = []
workers = []
fuel_cells = []
mod_type = ''
action = ''
move = 1

# Procedure declarations

typing_speed = 100
def write(t):
    for l in t:
        sys.stdout.write(l)
        sys.stdout.flush()
        time.sleep(random.random()*10.0/typing_speed)
    print('')

def load_module():
  global module, possible_moves
  possible_moves = get_modules_from(module)
  output_module()

def get_modules_from(module):
  global mod_type
  moves = []
  text_file = open('module' + str(module) + '.txt', 'r')
  mod_type = text_file.readline()
  for counter in range(0, 4):
    move_read = text_file.readline()
    move_read = int(move_read.strip())
    if move_read != 0:
      moves.append(move_read)
  text_file.close()
  return moves

def output_module():
  global module, mod_type
  print()
  print('--------------------------------------------------------------')
  print()
  message = 'You are in module ' + str(module) +  ' - This is the ' +  str(mod_type)
  write(message)
  check_npcs()
  print()

def check_npcs():
  global module, fuel, queen, vent_shafts, fuel_cells, workers, info_panels, power
  if module == queen:
    move_queen()
  elif module in vent_shafts:
    write('There is a bank of fuel cells in here.')
    write('You load one into your flame thrower.')
    fuel_gained = random.randint(2, 5) * 10
    fuel = fuel + fuel_gained
    message = 'Fuel now reading ' + str(fuel)
    write(message)
    write('The doors suddenly lock.')
    write('You climb into the ventilation shaft.')
    write('You follow the passage and find yourself sliding into a module.')
    clear_output()
    last_module = module
    module = random.randint(1, num_modules)
    while module == last_module or module in vent_shafts:
      module = random.randint(1, num_modules)
    load_module()
  elif module in info_panels:
    write('There is an information panel in this module.')
    print('1. Interact')
    print('2. Ignore')
    choice = 0
    while choice not in [1, 2]:
      try:
        choice = int(input('> '))
      except ValueError:
        write('Invalid')
    if power > 50:
      if choice == 1:
        message = 'The queen is in module ' + str(queen)
        write(message)
        power -= 50
    else:
      if choice == 1:
        write('There is not enough power. The panel is dead.')
  elif module in workers:
    write('There is a worker alien in this module.')
  elif module in fuel_cells:
    write('There is a fuel cell in this module.')
    write('You load it into your flamethrower.')
    fuel_gained = random.randint(2, 5) * 10
    fuel = fuel + fuel_gained
    message = 'You now have ' + str(fuel) + ' fuel.'
    write(message)

def output_moves():
  global possible_moves
  print()
  write('From here you can move to modules: ')
  for move in possible_moves:
    message = str(move) + ' '
    write(message)
  print()

def lock():
  global num_modules, power, locked
  valid = False
  while not valid:
    try:
      new_lock = int(input('Enter module to lock: '))
    except ValueError:
      write('Invalid')
    else:
      valid = True
  if new_lock < 0 or new_lock > num_modules:
    write('Invalid module. Operation failed.')
  elif new_lock == queen:
    write('Operation failed. Unable to lock module.')
  elif new_lock == locked:
    write('Module already locked.')
  else:
    locked = new_lock
    message = 'Aliens cannot get into module ' + str(locked)
    write(message)
  power_used = 25 + 5 * random.randint(0, 5)
  power -= power_used
  input()

def move_queen():
  global num_modules, moduule, last_module, locked, queen, won, vent_shafts, killed
  if module == queen:
    write('There it is! The queen alien is in this module...')
    moves_to_make = random.randint(1, 3)
    can_move_to_last_module = False
    while moves_to_make > 0:
      escapes = get_modules_from(queen)
      if module in escapes:
        escapes.remove(module)
      if last_module in escapes and can_move_to_last_module == False:
        escapes.remove(last_module)
      if locked in escapes:
        escapes.remove(locked)
      if len(escapes) == 0:
        moves_to_make = 0
        write("...and the door is locked. It's trapped.")
        if fuel >= 100:
          won = True
        else:
          killed = True
      else:
        if moves_to_make == 1:
          write("...and has escaped.")
        queen = random.choice(escapes)
        moves_to_make -= 1
        can_move_to_last_module = True
        while queen in vent_shafts:
          if moves_to_make > 1:
            write("...and has escaped.")
          write('We can hear scuttling in the vent shafts.')
          valid_move = False
          while not valid_move:
            valid_move = True
            queen = random.randint(1, num_modules)
            if queen in vent_shafts:
              valid_move = False
          moves_to_make = 0

def intuition():
  global possible_moves, workers, vent_shafts, queen, info_panels
  sound = False
  feel = False
  near_panel = False
  for connected_module in possible_moves:
    if connected_module in workers:
      sound = True
    if connected_module in vent_shafts:
      feel = True
    if connected_module == queen:
      write('Listen...Did you hear that?')
    if connected_module in info_panels:
      near_panel = True
  if feel == True:
    write('I can feel cold air.')
  if sound == True:
    write('I can hear scuttling.')
  if near_panel == True:
    write('There should be a panel near here. We could use it to find the queen.')

def get_action():
  global module, last_module, possible_moves, action, move, power, vent_shafts, queen, workers, info_panels, fuel_cells
  print('1. Move')
  print('2. Use scanner')
  choice = 0
  while choice not in [1, 2]:
    try:
      choice = int(input('> '))
    except ValueError:
      write('Choice invalid')
  if choice == 1:
    valid = False
    move = 0
    while move not in possible_moves:
      try:
        move = int(input('Enter module to move to: '))
      except ValueError:
        move = int(input('Enter module to move to: '))
    last_module = module
    module = move
  else:
    print()
    print('1. Power')
    print('2. Lock')
    print('3. Lifeforms')
    print('4. Scan')
    option = 0
    while option not in [1, 2, 3, 4]:
      try:
        option = int(input('> '))
      except ValueError:
        write('Choice invalid')
    if option == 1:
      message = 'Power level: ' + str(power)
      write(message)
      input()
    elif option == 2:
      write('Scanner ready. Press enter to lock module.')
      input()
      lock()
    elif option == 3:
      lifeforms = len(workers) + 1
      message = 'Lifeforms detected: ' + str(lifeforms)
      write(message)
      input()
    elif option == 4:
      if power < 26:
        write('Not enough power left.')
      else:
        scan_module = 0
        while scan_module not in possible_moves:
          try:
            scan_module = int(input('Enter module to scan: '))
          except ValueError:
            write('Invalid')
        if scan_module in vent_shafts:
          write('There is a vent shaft in this module.')
        elif scan_module in workers:
          write('There is a worker in this module.')
        elif scan_module in info_panels:
          write('There is an info panel in this module.')
        elif scan_module in fuel_cells:
          write('There is a fuel cell in this module.')
        elif scan_module == queen:
          write('The queen is in this module.')
        else:
          write('There is nothing special in this module.')
        input()

def spawn_npcs():
  global num_modules, queen, vent_shaft, info_panels, workers
  module_set = []
  for counter in range(2, num_modules + 1):
    module_set.append(counter)
  random.shuffle(module_set)
  i = 0
  queen = module_set[i]
  for counter in range(0, 3):
    i = i + 1
    vent_shafts.append(module_set[i])
  for counter in range(0, 2):
    i = i + 1
    info_panels.append(module_set[i])
  for counter in range(0, 3):
    i = i + 1
    workers.append(module_set[i])
  for counter in range(0, 2):
    i = i + 1
    fuel_cells.append(module_set[i])

def play_game():
  global power, alive, won, killed
  spawn_npcs()
  print('Queen alien is located in module: ', queen)
  print('Ventilation shafts are located in modules: ', vent_shafts)
  print('Information panels are located in modules: ', info_panels)
  print('Worker aliens are located in modules: ', workers)
  while alive and not won:
    if killed == True:
      break
    load_module()
    power -= 1
    if power == 0:
      alive == False
    if not won and alive:
      if killed == True:
        break
      if power < 101:
        write('Life support will shut off any minute now.')
      if fuel <= 100:
        write('Low fuel!')
      intuition()
      output_moves()
      get_action()
      clear_output()
  if killed == True:
    write('The alien lunges at you and kills you.')
    write('Game over. You lost!')
    input('Press enter to return to menu.')
    clear_output()
  elif won == True:
    write('The queen is trapped and you burn it to death with your flamethrower.')
    write('Game over. You won!')
    input('Press enter to return to menu.')
    clear_output()
  elif alive == False:
    write('The station has run out of power. Unable to sustain life support, you die.')
    write('Game over. You lost!')
    input('Press enter to return to menu.')
    clear_output()

def output_story():
  clear_output()
  write('A bacterial species was discoverd on Mars and brought to the space station Charles Darwin for testing.')
  write('After incubating it in conditions similiar to ancient Mars, the scientists discovered it began to grow.')
  write('It grew by feeding on the other bacteria and day by day its size and strength grew.')
  write('Two days ago the alien killed a scientist and escaped into the depths of the Charles Darwin.')
  write('You are now trapped on the Charles Darwin with a dangerous alien.')
  write('The alien damaged the life support system. After a while the astronauts wil die.')
  write('Help will only arrive when the alien is killed.')
  write('Time is running out...')
  input('Press enter to continue... ')
  clear_output()

def instructions():
  clear_output()
  write('The alien must be trapped in a module and killed with a flamethrower before life support runs out.')
  write('You have 1000 power to start with. Every turn this will decrease by 1 and when it reaches 0, you die.')
  write("Using your scanner or information panels also depletes the station's power.")
  write('You can move from module to module to find the alien. You can also lock modules to trap the alien or use a scanner to find it.')
  write('If you enter a module with a vent the doors will lock and you will have to take the vent to a random module.')
  write('You may encounter a worker aliens. You can choose to frighten them or kill them using your flamethrower.')
  write('If you fail to kill/frighten them they will kill you.')
  write('You will be able to pick up fuel for your flamethrower in certain modules.')
  write('Good luck, astronaut!')
  input('Press enter to continue... ')
  clear_output()

def design():
  print('-------  |------  |       ------- |        | |\\         /|')
  time.sleep(0.5)
  print('   |     |        |          |    |        | | \\       / |')
  time.sleep(0.5)
  print('   |     |------  |          |    |        | |  \\     /  |')
  time.sleep(0.5)
  print('   |     |        |          |    |        | |   \\   /   |')
  time.sleep(0.5)
  print('   |     |------  |------ ------- \\--------/ |    \\ /    |')
  print()
  time.sleep(0.5)

# Main program

while True:
  design()
  print('1. Play')
  print('2. Story')
  print('3. Instructions')
  print('4. Quit')
  option = 0
  while option not in [1, 2, 3, 4]:
    try:
      option = int(input('> '))
    except ValueError:
      print('Select a valid option.')
  if option == 1:
    clear_output()
    play_game()
  elif option == 2:
    output_story()
  elif option == 3:
    instructions()
  else:
    break
clear_output()


--------------------------------------------------------------

You are in module 1 - This is the entry module


I can hear scuttling.

From here you can move to modules: 
2 
16 

1. Move
2. Use scanner


KeyboardInterrupt: Interrupted by user